In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import imageio
import skimage
import os,time
import pickle

In [ ]:
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

In [ ]:
tf.set_random_seed(42)

In [ ]:
logging = tf.logging
logging.set_verbosity(logging.INFO)

def log_msg(msg):
   logging.info(f'{time.ctime()}: {msg}')

In [ ]:
num_images = 1000
num_classes = 10
folder_path = '../data/filtered_train'
label_path = '../data/filtered_train.csv'
batch_size = 128
valid_folder = '../data/filtered_valid'
valid_label = '../data/filtered_valid.csv'

In [ ]:
def cifar_dataset(image_folder_path, label_file, b_size, num_images):
#     images
    all_images = []
    for i in range(num_images):
        image_path = image_folder_path + '/' + str(i) + '.png'
        img = imageio.imread(image_path)
        img = (img - img.mean()) / img.std()
        all_images.append(img)
        if i%1000 == 0:
            print("Processed " + str(i))
        
    all_images = np.array(all_images)
    all_images = all_images.reshape((-1,256,256,3))
    
    dataset = tf.data.Dataset.from_tensor_slices((all_images)).batch(b_size)
    
#     labels
    lf = open(label_file,'r')
    labels = lf.read().split('\n')
    labels.remove('')
    labels = list(map(int, labels))
    print(len(labels))
    labels = labels[:num_images]
    labels = np.array(labels)
    all_labels = tf.data.Dataset.from_tensor_slices((labels)).batch(b_size)
            
    return dataset, all_labels  

In [ ]:
class Convolution(tf.keras.Model):
    def __init__(self, filters, size, stride, padding, activation, initializer ):
        super(Convolution, self).__init__()
        self.conv = tf.layers.Conv2D(filters=filters, kernel_size=size, strides=stride, padding=padding, activation=activation, kernel_initializer=initializer)
#         self.conv = tf.layers.Conv2D(filters=filters, kernel_size=size, strides=stride, padding=padding, activation=activation)
    
    def call(self, inp):
        return self.conv(inp)
        

In [ ]:
class MaxPool(tf.keras.Model):
    def __init__(self, size, stride, padding):
        super(MaxPool, self).__init__()
        self.pool = tf.layers.MaxPooling2D(pool_size = size, strides =stride, padding = padding)
    
    def call(self, inp):
        return self.pool(inp)

In [ ]:
def lrn(x, radius, alpha, beta, bias=1.0):
#     return tf.nn.local_response_normalization(x, depth_radius = radius, alpha = alpha, beta = beta, bias = bias)
    return tf.nn.lrn(x, depth_radius = radius, alpha = alpha, beta = beta, bias = bias)

In [ ]:
class AlexNet(tf.keras.Model):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv1 = Convolution(96,11,4,'VALID',tf.nn.relu, tf.truncated_normal_initializer(stddev=0.01))
        self.pool1 = MaxPool(3,2, 'VALID')
        
        self.conv2 = Convolution(256,5,1,'SAME',tf.nn.relu, tf.truncated_normal_initializer(stddev=0.01))
        self.pool2 = MaxPool(3,2, 'VALID') 
        
        self.conv3 = Convolution(384,3,1,'SAME',tf.nn.relu, tf.truncated_normal_initializer(stddev=0.01))
        
        self.conv4 = Convolution(384,3,1,'SAME',tf.nn.relu, tf.truncated_normal_initializer(stddev=0.01))
        
        self.conv5 = Convolution(256,3,1,'SAME',tf.nn.relu, tf.truncated_normal_initializer(stddev=0.01))
        self.pool5 = MaxPool(3,2,'VALID')
        
#         to be replaced with attention
        self.fc6 = tf.layers.Dense(4096, activation = tf.nn.relu)
        self.fc7 = tf.layers.Dense(4096, activation = tf.nn.relu)
        self.fc8 = tf.layers.Dense(num_classes, activation = None)
    
    def call(self, image):
        conv1 = self.conv1(image)
        pool1 = self.pool1(conv1)
        norm1 = lrn( tf.cast(pool1, dtype = tf.float32), 2, 2e-05, 0.75)
        
        conv2 = self.conv2(norm1)
        pool2 = self.pool2(conv2)
        norm2 = lrn( tf.cast(pool2, dtype = tf.float32), 2, 2e-05, 0.75)
        
        conv3 = self.conv3(norm2)
        
        conv4 = self.conv4(conv3)
        
        conv5 = self.conv5(conv4)
        pool5 = self.pool5(conv5)

        # to be replaced        
        fc6 = self.fc6(tf.layers.flatten(pool5))
        fc6 = tf.nn.dropout(fc6, keep_prob = 0.5)
        fc7 = self.fc7(fc6)
        fc7 = tf.nn.dropout(fc7, keep_prob = 0.5)
        fc8 = self.fc8(fc7)
          
        return fc8
        
    

In [ ]:
def prediction_loss_fun(model, data, labels):
    logits = model(data)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
#     return tf.reduce_sum(loss)/tf.cast(data.shape[0], dtype = tf.float32)
    return tf.reduce_mean(loss)

In [ ]:
def get_accuracy(model, data, labels):
    pred = tf.nn.softmax(model(data))
#     print("predicted..")
#     print(tf.argmax(pred, axis=1))
#     print("actual..")
#     print(labels)
    accuracy_val = tf.reduce_sum( tf.cast( tf.equal( tf.argmax(pred, axis=1), labels),dtype=tf.float32))/float(pred.shape[0].value)
    return accuracy_val

In [ ]:
def shuffle_data(data, label):
    idx = np.random.permutation(data.shape[0].value)
    
    datum = np.array(data)[idx]
    datum = tf.convert_to_tensor(datum)
    
    lab = np.array(label)[idx]
    lab = tf.convert_to_tensor(lab)
    
    return datum, lab

In [ ]:
dataset, labels = cifar_dataset(folder_path, label_path, batch_size, num_images)
valid_data, val_labels = cifar_dataset(valid_folder, valid_label, 300, 300)
val_data = next(iter(valid_data))
val_lab = next(iter(val_labels))

In [ ]:
anet = AlexNet(learning_rate = 1e-4)

In [ ]:
# opt = tf.train.AdamOptimizer(tf.train.inverse_time_decay(0.01, 0,1, 0.0001))
opt = tf.train.AdamOptimizer()
# opt = tf.train.MomentumOptimizer(learning_rate=1e-4, momentum=0.9)

In [ ]:
loss_and_grads_fun = tfe.implicit_value_and_gradients(prediction_loss_fun)

In [ ]:
checkpoint_dir = '../pure_alexnet_model'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
root = tfe.Checkpoint(optimizer=opt, model=anet, optimizer_step=tf.train.get_or_create_global_step())

In [ ]:
def get_valid_acc(model, dataset, labels):
    valid_acc = 0
    valid_loss = 0
    count = 0
    for datum,lab in zip(dataset,labels):
        count += 1
        valid_loss += prediction_loss_fun(anet, datum, lab).numpy()
        valid_acc += get_accuracy(anet, datum, lab).numpy()
    return valid_loss/count, valid_acc/count

In [ ]:
NUM_EPOCHS = 5
STATS_STEPS = 1

# valid_loss, acc = get_valid_acc(anet, valid_data, val_labels)
valid_loss = prediction_loss_fun(anet, val_data, val_lab).numpy()
acc = get_accuracy(anet, val_data, val_lab).numpy()*100
log_msg(f'Initial Valid loss: {valid_loss: 0.4f} accuracy: {acc: f}%')

for epoch_num in range(NUM_EPOCHS):
    print("Epoch : " + str(epoch_num))
    step_num = 0
    for data, label in zip(dataset, labels):
        step_num += 1
#         shuffle
        datum, lab = shuffle_data(data, label)
    
        loss_value, gradients = loss_and_grads_fun(anet, datum, lab)
        opt.apply_gradients(gradients, global_step=tf.train.get_or_create_global_step())
        
#         opt.minimize(lambda: prediction_loss_fun(anet, datum, lab), global_step = tf.train.get_or_create_global_step())        
        if (step_num % STATS_STEPS == 0) or (epoch_num == NUM_EPOCHS-1):
            print("Stat step " + str(step_num))
            loss = prediction_loss_fun(anet, datum, lab).numpy()
            accuracy = get_accuracy(anet, datum, lab).numpy()*100
            log_msg(f'Epoch: {epoch_num} Step: {step_num} Train loss: {loss: 0.4f} accuracy: {accuracy: f}%') 
#             loss, accuracy = get_valid_acc(anet, valid_data, val_labels)
            loss = prediction_loss_fun(anet, val_data, val_lab).numpy()
            accuracy = get_accuracy(anet, val_data, val_lab).numpy()*100
            log_msg(f'Epoch: {epoch_num} Step: {step_num} Valid loss: {loss: 0.4f} accuracy: {accuracy: f}%')
            if loss < valid_loss:
                print("Improvement in validation loss. Saving..")
                valid_loss = loss
                save_path = root.save(checkpoint_prefix)

In [ ]:
checkpoint_dir = '../pure_alexnet_final_model'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
root = tfe.Checkpoint(optimizer=opt, model=anet, optimizer_step=tf.train.get_or_create_global_step())
save_path = root.save(checkpoint_prefix)

In [ ]:
# check test accuracy
test_folder = '../data/filtered_test'
test_file = '../data/filtered_test.csv'
test_data, test_labels = cifar_dataset(test_folder, test_file, 500, 500)
test_data = next(iter(test_data))
test_lab = next(iter(test_labels))

In [ ]:
loss = prediction_loss_fun(anet, test_data, test_lab).numpy()
accuracy = get_accuracy(anet, test_data, test_lab).numpy()*100

In [ ]:
loss

In [ ]:
accuracy